# **CUDA 1**

*Implementation* of a **batched arbitrarily-size matrix multiplication** kernel.

Let dimensions be identical for all matrix multiplications in the batch.<br>
These being $m, k, n \in \mathbb{N}$.<br>
While $batch \in \mathbb{N}$ is the batch size.

You are provided as input the following matrices:<br>
$N_0, N_1, N_2, ... N_{batch - 1} \in \mathbb{M}^{k \times n}$<br>
$M \in \mathbb{M}^{m \times k}$

You need to compute:<br>
$P_0, P_1, P_2, ... P_{batch - 1} \in \mathbb{M}^{m \times n}$

Where $P_i = M \otimes N_i$ for each $i \in \{0, ..., batch - 1\}$.

---

To get a general performance metric rely on the profiler, specifically look for the `cuda_gpu_kern_sum` and try to minimize the `Total Time (ns)` of your kernel. You may also want to improve `cuda_gpu_mem_time_sum`.

Step one is beating the reference implementation, that should be easy, then you can use all tricks in the book to push it further.
Anything goes, but if you use "exotic" tricks we want an explanation.
In fact, submitting your work, be sure to fill out the [report](#report) with brief insights of what you did.

## **Colab Setup**

In [ ]:
%%capture
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install

In [ ]:
!mkdir /home/cuda
%cd /home/cuda

## **Code**

In [ ]:
%%writefile bmatmul.cpp
// DON'T CHANGE THIS ^^ FILENAME!
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// utility for wrapping CUDA API calls and log any error they may return (use this for debugging)
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
  if (code != cudaSuccess) {
    fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
    if (abort)
      exit(code);
  }
}

// === DO NOT CHANGE THIS ===
// host-side version, used to validate results
__host__
void batchedMatMulHost(float* M, float* N, float* P, int m, int k, int n, int batch) {
  for (int b = 0; b < batch; b++) {
    for (int row = 0; row < m; row++) {
      for (int col = 0; col < n; col++) {
        float value = 0.0f;
        for (int i = 0; i < k; i++) {
          float a = M[row*k + i];
          float c = N[b*(k*n) + i*n + col];
          value += a * c;
        }
        P[b*(m*n) + row*n + col] = value;
      }
    }
  }
}

void initWith(float number, float* arr, int size) {
  for (int i = 0; i < size; i++)
    arr[i] = number;
}

void initRandom(float* arr, int size, unsigned int seed, float minVal = 0.0f, float maxVal = 1.0f) {
  srand(seed);
  for (int i = 0; i < size; i++) {
    float r = (float)rand() / RAND_MAX;
    arr[i] = minVal + r * (maxVal - minVal);
  }
}

void checkResult(float* arr1, float* arr2, int size) {
  const float atol = 1e-4f; // absolute tolerance for fp32 (lack of) associativity
  const float rtol = 1e-4f; // relative tolerance for fp32 (lack of) associativity
  for (int i = 0; i < size; i++) {
    float diff = fabs(arr1[i] - arr2[i]);
    float tol = atol + rtol*fabs(arr2[i]);
    if (diff > tol) {
      printf("Error at %d: %f != %f (diff=%e, tol=%e)\n", i, arr1[i], arr2[i], diff, tol);
      exit(1);
    }
  }
}
// ==========================

#define TILE_WIDTH 16
#define BLOCK_WIDTH 16
#define BLOCK_HEIGHT 16

__global__ void batchedMatMul(
        const float * __restrict__ M,
        const float * __restrict__ N,
        float * const __restrict__ P,
        int m, int k, int n, int batch) {
    __shared__ float Mds[TILE_WIDTH*2][TILE_WIDTH];
    __shared__ float Nds[TILE_WIDTH][TILE_WIDTH*2];

    int bx = blockIdx.x, by = blockIdx.y, b = blockIdx.z;
    int tx = threadIdx.x, ty = threadIdx.y;

    int Row = by * BLOCK_HEIGHT * 2 + ty * 2;
    int Col = bx * BLOCK_WIDTH * 2 + tx * 2;

    float P00 = 0.0f, P01 = 0.0f, P10 = 0.0f, P11 = 0.0f;

    for (int t = 0; t < (k + TILE_WIDTH - 1) / TILE_WIDTH; ++t) {
        int tiledCol = t * TILE_WIDTH + tx;
        int tiledRow = t * TILE_WIDTH + ty;

        // Ogni thread carica due righe di M e due colonne di N
        Mds[ty*2][tx] = ((Row < m && tiledCol < k) ? M[Row * k + tiledCol] : 0.0f);
        Mds[ty*2 + 1][tx] = ((Row + 1 < m && tiledCol < k) ? M[(Row + 1) * k + tiledCol] : 0.0f);

        Nds[ty][tx*2] = ((Col < n && tiledRow < k) ? N[b * k * n + tiledRow * n + Col] : 0.0f);
        Nds[ty][tx*2 + 1] = ((Col + 1 < n && tiledRow < k) ? N[b * k * n + tiledRow * n + Col + 1] : 0.0f);

        __syncthreads();

        for (int i = 0; i < TILE_WIDTH; ++i) {
            float a0 = Mds[ty*2][i];
            float a1 = Mds[ty*2 + 1][i];
            float b0 = Nds[i][tx*2];
            float b1 = Nds[i][tx*2 + 1];

            P00 += a0 * b0;
            P01 += a0 * b1;
            P10 += a1 * b0;
            P11 += a1 * b1;
        }

        __syncthreads();
    }

    if (Row < m && Col < n) {
        P[b * m * n + Row * n + Col] = P00;
    }
    if (Row < m && Col + 1 < n) {
        P[b * m * n + Row * n + Col + 1] = P01;
    }
    if (Row + 1 < m && Col < n) {
        P[b * m * n + (Row + 1) * n + Col] = P10;
    }
    if (Row + 1 < m && Col + 1 < n) {
        P[b * m * n + (Row + 1) * n + Col + 1] = P11;
    }

}


int main(int argc, char** argv) {
  // === DO NOT CHANGE THIS ===
  if (argc != 6) {
    printf("Usage: %s <m> <k> <n> <batch> <seed>\n", argv[0]);
    exit(1);
  }

  int m = atoi(argv[1]); // rows of Ms and Ps
  int k = atoi(argv[2]); // cols of Ms, rows of Ns
  int n = atoi(argv[3]); // cols of Ns and Ps
  int batch = atoi(argv[4]); // number of matrix pairs
  unsigned int seed = (unsigned int)atoi(argv[5]); // seed for random initialization

  printf("Running batched matmul with m=%d, k=%d, n=%d, batch=%d, seed=%u\n", m, k, n, batch, seed);

  const int sizeM = m*k;
  const int sizeN = k*n*batch;
  const int sizeP = m*n*batch;
  // ==========================
  // initialize
  float *M, *N, *P;
  gpuErrchk(cudaMallocHost((void**)&M, sizeM * sizeof(float)));
  gpuErrchk(cudaMallocHost((void**)&N, sizeN * sizeof(float)));
  gpuErrchk(cudaMallocHost((void**)&P, sizeP * sizeof(float)));

  initRandom(M, sizeM, seed);
  initRandom(N, sizeN, seed + 1);
  initWith(0.0f, P, sizeP);
  // ==========================

  // here, you can change anything
  float *M_d;
  float *N_d;
  float *P_d;

  gpuErrchk(cudaMalloc((void**)&M_d, sizeM * sizeof(float)));
  gpuErrchk(cudaMalloc((void**)&N_d, sizeN * sizeof(float)));
  gpuErrchk(cudaMalloc((void**)&P_d, sizeP * sizeof(float)));

  gpuErrchk(cudaMemcpy(M_d, M, sizeM * sizeof(float), cudaMemcpyHostToDevice));
  gpuErrchk(cudaMemcpy(N_d, N, sizeN * sizeof(float), cudaMemcpyHostToDevice));
  gpuErrchk(cudaMemcpy(P_d, P, sizeP * sizeof(float), cudaMemcpyHostToDevice));

  //dim3 numBlocks((n + blockSize.x - 1) / blockSize.x, (m + blockSize.y - 1) / blockSize.y);
  dim3 blockSize(BLOCK_WIDTH, BLOCK_HEIGHT);
  dim3 gridSize((n + BLOCK_WIDTH * 2 - 1) / (BLOCK_WIDTH * 2), (m + BLOCK_HEIGHT * 2 - 1) / (BLOCK_HEIGHT * 2), batch);

  batchedMatMul<<<gridSize, blockSize>>>(M_d, N_d, P_d, m, k, n, batch);
  gpuErrchk(cudaDeviceSynchronize());

  gpuErrchk(cudaMemcpy(P, P_d, sizeP * sizeof(float), cudaMemcpyDeviceToHost));

  // === DO NOT CHANGE THIS ===
  // However: once you know results are correct, you can temporarily
  //          comment this out if you want to test performance on large
  //          matrices, since the evaluation on CPU can get pretty slow.
  printf("Checking results on CPU...\n");
  float* P_host = (float*)malloc(sizeP * sizeof(float));
  initWith(0.0f, P_host, sizeP);
  batchedMatMulHost(M, N, P_host, m, k, n, batch);
  checkResult(P, P_host, m*n*batch);
  printf("All results matched, success!");
  // ==========================

  // here, you can change anything, e.g. add some logging
  gpuErrchk(cudaFree(M_d));
  gpuErrchk(cudaFree(N_d));
  gpuErrchk(cudaFree(P_d));

  cudaFreeHost(M);
  cudaFreeHost(N);
  cudaFreeHost(P);
  free(P_host);

  return 0;
}


## **Compile, Run, Profile**

Compile and run:

In [ ]:
!pwd
!mv bmatmul.cpp bmatmul.cu
!nvcc -arch=sm_75 bmatmul.cu -o bmatmul

In [ ]:
!./bmatmul 123 456 678 3 1

Profile:

In [ ]:
!nsys profile --stats=true ./bmatmul 256 512 384 16 119

<a name="report"></a>
## **Brief Report**


**TEAM information:**
- member-1: GIORGIO, MANTOAN
- member-2: ANDREA, OGGIONI
- member-3: MATTEO, PARIMBELLI


- Used shared memory to reduce global memory access (shared is faster than global).
- Use pinned memory to speedup memory transfers.
- Multiplication is performed by tiling.
- Batching implemented using the `z` coordinate of the grid.
- Use of micro tiling to improve speed (more math per memory access).
- Use of `const` and `__restrict__` pointers where possible to encourage the compiler to perform further minor optimizations.

- Tried `#pragma unroll` on the inner for-loop but kernel was 2$\mu$s slower.
